In [ ]:
# default_exp complete

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Complete

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
all_pivot_selected_folder  = "D:/data/parq_pivot_select"
all_pivoted_folder = "D:/data/parq_pivot_split"
all_processed_folder = "D:/data/parq_processed/"
all_data_local_folder = "./data/"

pivot_group = ["cik","ticker","adsh","period","filed","form","qtrs","fp"]
analyze_fields = ["value_count","null_count","difference"]

## 00_Tools

In [ ]:
def count_null(df: pd.DataFrame):
    columns = list(df.columns)
    value_cols = set(columns) -  set(pivot_group)
    value_cols = value_cols -  set(analyze_fields)
    
    df['value_count'] = df[value_cols].notnull().sum(axis=1)
    df['null_count'] = df[value_cols].isnull().sum(axis=1)
    
    print("lines with missing values: ", df[df.null_count > 0].shape[0])

In [ ]:
def load_data(stmt:str) -> pd.DataFrame:
    df = pd.read_csv(all_processed_folder + stmt + "_not_cleaned.csv")
    df.period = pd.to_datetime(df.period)
    df.filed = pd.to_datetime(df.filed)
    
    return df

In [ ]:
def check_number_of_reports(df: pd.DataFrame):
    """
    check if there is only one entry per report (unique adsh)
    """
    print("is there just one report per ADSH: ", len(df.adsh.unique()) == df.shape[0])

In [ ]:
def missing_value_report(df: pd.DataFrame):
    df_missing = df[df.null_count > 0]
    print(df_missing.shape[0])
    sns.catplot(x='null_count', kind='count', data=df_missing)

## 01_Complete BS

### load data

In [ ]:
bs_df = load_data("bs")
count_null(bs_df)

lines with missing values:  1301


In [ ]:
bs_df.shape

(116519, 18)

In [ ]:
check_number_of_reports(bs_df)

is there just one report per ADSH:  True


### complete missing Equity

In [ ]:
# complete Equity_hj = Assets - Liabilities
do_set_equity = (bs_df.Equity_hj.isnull()  & bs_df.Assets.notnull() & bs_df.Liabilities.notnull())
bs_df.loc[do_set_equity, 'Equity_hj'] = bs_df.loc[do_set_equity, 'Assets'] - bs_df.loc[do_set_equity, 'Liabilities']

In [ ]:
count_null(bs_df)

lines with missing values:  559


### complete missing liabilities

In [ ]:
do_set_liab = (bs_df.Assets.notnull()  & bs_df.Equity_hj.notnull()  & bs_df.Liabilities.isnull() & bs_df.LiabilitiesCurrent.isnull() & bs_df.LiabilitiesNoncurrent.isnull())
bs_df.loc[do_set_liab, 'Liabilities'] = bs_df.loc[do_set_liab, 'Assets'] - bs_df.loc[do_set_liab, 'Equity_hj']
bs_df.loc[do_set_liab, 'LiabilitiesCurrent'] = bs_df.loc[do_set_liab, 'Liabilities']
bs_df.loc[do_set_liab, 'LiabilitiesNoncurrent'] = 0.0

In [ ]:
count_null(bs_df)

lines with missing values:  282


### complete missing assets

In [ ]:
do_set_liab = (bs_df.Liabilities.notnull()  & bs_df.Equity_hj.notnull()  & bs_df.Assets.isnull() & bs_df.AssetsCurrent.isnull() & bs_df.AssetsNoncurrent.isnull())
bs_df.loc[do_set_liab, 'Assets'] = bs_df.loc[do_set_liab, 'Liabilities'] + bs_df.loc[do_set_liab, 'Equity_hj']
bs_df.loc[do_set_liab, 'AssetsCurrent'] = bs_df.loc[do_set_liab, 'Liabilities']
bs_df.loc[do_set_liab, 'AssetsNoncurrent'] = 0.0

In [ ]:
count_null(bs_df)

lines with missing values:  274


### save

In [ ]:
fields_to_drop = set(analyze_fields).intersection(set(bs_df.columns))
bs_df.drop(columns=fields_to_drop).to_csv(all_data_local_folder + "05_bs_completed.csv", index=False)

### remove companies with incomplete balancesheets

In [ ]:
incomplete_ciks = bs_df[bs_df.null_count > 3].cik.unique()
print(len(incomplete_ciks))

82


In [ ]:
bs_cleaned = bs_df[~bs_df.cik.isin(incomplete_ciks)]

In [ ]:
bs_cleaned.shape

(114481, 18)

In [ ]:
# save them
fields_to_drop = set(analyze_fields).intersection(set(bs_df.columns))
bs_cleaned.drop(columns=fields_to_drop).to_csv(all_processed_folder + "bs_clean.csv", index=False)

## 02_Complete CF

### load data

In [ ]:
cf_df = load_data("cf")
count_null(cf_df)

lines with missing values:  1359


In [ ]:
cf_df.shape

(116187, 18)

In [ ]:
check_number_of_reports(cf_df)

is there just one report per ADSH:  True


### complete CashIncreaseDecrease

In [ ]:
# per definition CashIncreaseDecrease = NetCashProvidedByUsedInOperatingActivities + NetCashProvidedByUsedInInvestingActivities + NetCashProvidedByUsedInFinancingActivities
do_set_cshinc = (cf_df.CashIncreaseDecrease_hj.isnull()  & cf_df.NetCashProvidedByUsedInOperatingActivities.notnull()  & 
                 cf_df.NetCashProvidedByUsedInInvestingActivities.notnull() & cf_df.NetCashProvidedByUsedInFinancingActivities.notnull())
cf_df.loc[do_set_cshinc, 'CashIncreaseDecrease_hj'] = cf_df.loc[do_set_cshinc, 'NetCashProvidedByUsedInOperatingActivities'] \
                                                    + cf_df.loc[do_set_cshinc, 'NetCashProvidedByUsedInInvestingActivities'] \
                                                    + cf_df.loc[do_set_cshinc, 'NetCashProvidedByUsedInFinancingActivities'] 

In [ ]:
count_null(cf_df)

lines with missing values:  377


### complete NetCashProvidedByUsedInOperatingActivities

In [ ]:
# per definition CashIncreaseDecrease = NetCashProvidedByUsedInOperatingActivities + NetCashProvidedByUsedInInvestingActivities + NetCashProvidedByUsedInFinancingActivities
# hence NetCashProvidedByUsedInOperatingActivities = CashIncreaseDecrease - (NetCashProvidedByUsedInInvestingActivities + NetCashProvidedByUsedInFinancingActivities)
do_set_netop = (cf_df.CashIncreaseDecrease_hj.notnull()  & cf_df.NetCashProvidedByUsedInOperatingActivities.isnull()  & 
                 cf_df.NetCashProvidedByUsedInInvestingActivities.notnull() & cf_df.NetCashProvidedByUsedInFinancingActivities.notnull())
cf_df.loc[do_set_netop, 'NetCashProvidedByUsedInOperatingActivities'] = cf_df.loc[do_set_netop, 'CashIncreaseDecrease_hj'] \
                                                                      - (cf_df.loc[do_set_netop, 'NetCashProvidedByUsedInInvestingActivities']  + cf_df.loc[do_set_netop, 'NetCashProvidedByUsedInFinancingActivities'] )

In [ ]:
count_null(cf_df)

lines with missing values:  316


### save

In [ ]:
fields_to_drop = set(analyze_fields).intersection(set(cf_df.columns))
cf_df.drop(columns=fields_to_drop).to_csv(all_data_local_folder + "05_cf_completed.csv", index=False)

### remove companies with incomplete cash flow statment

In [ ]:
incomplete_ciks = cf_df[cf_df.null_count > 1].cik.unique()
print(len(incomplete_ciks))

146


In [ ]:
cf_cleaned = cf_df[~cf_df.cik.isin(incomplete_ciks)]

In [ ]:
cf_cleaned.shape

(112417, 18)

In [ ]:
fields_to_drop = set(analyze_fields).intersection(set(cf_df.columns))
cf_cleaned.drop(columns=fields_to_drop).to_csv(all_processed_folder + "cf_clean.csv", index=False)

## 03_Complete IS

### load data

In [ ]:
is_df = load_data("is")
count_null(is_df)

lines with missing values:  115883


In [ ]:
is_df.shape

(115899, 21)

In [ ]:
check_number_of_reports(is_df)

is there just one report per ADSH:  True


### complete GrossProfit

In [ ]:
# per definition Grossprofit = Revenues - CostOfRevenue
do_set_grossp = (is_df.Revenues_hj.notnull()  & is_df.CostOfRevenue_hj.notnull()  & is_df.GrossProfit.isnull())
is_df.loc[do_set_grossp, 'GrossProfit'] = is_df.loc[do_set_grossp, 'Revenues_hj'] - is_df.loc[do_set_grossp, 'CostOfRevenue_hj']

In [ ]:
# per definition: grossprofit - opexpenses = op-income -> grossprofit = opincome + opexpenses
do_set_grossp1 = (is_df.OperatingExpenses.notnull()  & is_df.OperatingIncomeLoss_hj.notnull()  & is_df.GrossProfit.isnull())
is_df.loc[do_set_grossp1, 'GrossProfit'] = is_df.loc[do_set_grossp1, 'OperatingIncomeLoss_hj'] + is_df.loc[do_set_grossp1, 'OperatingExpenses']

### complete Revenues

In [ ]:
# per definition Grossprofit = Revenues - CostOfRevenue -> Revenues = Grossprofit + CostOfRevenue
do_set_rev = (is_df.Revenues_hj.isnull()  & is_df.CostOfRevenue_hj.notnull()  & is_df.GrossProfit.notnull())
is_df.loc[do_set_rev, 'Revenues_hj'] = is_df.loc[do_set_rev, 'CostOfRevenue_hj'] - is_df.loc[do_set_rev, 'GrossProfit']
set_to_zero_if_null(is_df,'Revenues_hj')

### complete CostOfRevenue

In [ ]:
# per definition Grossprofit = Revenues - CostOfRevenue -> CostOfRevenue = Revenues - Grossprofit
do_set_cstrev = (is_df.Revenues_hj.notnull()  & is_df.CostOfRevenue_hj.isnull()  & is_df.GrossProfit.notnull())
is_df.loc[do_set_cstrev, 'CostOfRevenue_hj'] = is_df.loc[do_set_cstrev, 'Revenues_hj'] - is_df.loc[do_set_cstrev, 'GrossProfit']

### complete OperatingIncomeLoss

In [ ]:
# per definition: opIncomeLoss = grossprofit - opexpenses
do_set_opinc = (is_df.OperatingExpenses.notnull()  & is_df.GrossProfit.notnull()  & is_df.OperatingIncomeLoss_hj.isnull())
is_df.loc[do_set_opinc, 'OperatingIncomeLoss_hj'] = is_df.loc[do_set_opinc, 'GrossProfit'] - is_df.loc[do_set_opinc, 'OperatingExpenses']

In [ ]:
(is_df.OperatingIncomeLoss_hj.isnull()).sum()

3903

### estimate OperatingIncomeLoss_hj based on GrossProfit and NetIncomeLoss

In [ ]:
# if OperatingIncomeLoss is not set, we can try to estimate it based on the average compared to GrossProfit and NetIncomeLoss

# based on existing OperatingIncomeLoss_hj Values, we calculate the average OperatingIncomeLoss_hj between GrossProfit and NetIncomeLoss (in percent)
#  we may only consider entries where NetIncomeLoss isn't equal to GrossProfit. since this would cause a division by zero
calc_mean_opinc_avg = (is_df.GrossProfit.notnull()  & is_df.NetIncomeLoss_hj.notnull()  & is_df.OperatingIncomeLoss_hj.notnull() & (is_df.NetIncomeLoss_hj != is_df.GrossProfit))
selected_cols = is_df[calc_mean_opinc_avg][['NetIncomeLoss_hj','GrossProfit','OperatingIncomeLoss_hj']].copy()

avg_opinc = ((selected_cols.GrossProfit-selected_cols.OperatingIncomeLoss_hj)/(selected_cols.GrossProfit-selected_cols.NetIncomeLoss_hj)).mean()

# if we have grossprofit and NetIncomeLoss_hj, we try estimate OperatingIncomeLoss based on the average position
do_updated_opinc = (is_df.GrossProfit.notnull() & is_df.OperatingIncomeLoss_hj.isnull() & is_df.NetIncomeLoss_hj.notnull())

is_df.loc[do_updated_opinc,'OperatingIncomeLoss_hj'] = is_df.loc[do_updated_opinc,'GrossProfit'] \
                                    - avg_opinc * (is_df.loc[do_updated_opinc,'GrossProfit'] - is_df.loc[do_updated_opinc,'NetIncomeLoss_hj'])

In [ ]:
(is_df.OperatingIncomeLoss_hj.isnull()).sum()

3597

### estimate OperatingIncomeLoss_hj based on Revenues and NetIncomeLoss

In [ ]:
# if OperatingIncomeLoss is not set, we can try to estimate it based on the average compared to Revenues and NetIncomeLoss

# based on existing OperatingIncomeLoss_hj Values, we calculate the average OperatingIncomeLoss_hj between Revenue and NetIncomeLoss (in percent)
calc_mean_opinc_avg = ((is_df.Revenues_hj > 0.0)  & (is_df.NetIncomeLoss_hj.notnull()  & is_df.OperatingIncomeLoss_hj.notnull()))
selected_cols = is_df[calc_mean_opinc_avg][['NetIncomeLoss_hj','Revenues_hj','OperatingIncomeLoss_hj']].copy()

avg_opinc = ((selected_cols.Revenues_hj-selected_cols.OperatingIncomeLoss_hj)/(selected_cols.Revenues_hj-selected_cols.NetIncomeLoss_hj)).mean()

# if we have revenue and NetIncomeLoss_hj, we try estimate OperatingIncomeLoss based on the average position
do_updated_opinc = (is_df.Revenues_hj.notnull() & is_df.OperatingIncomeLoss_hj.isnull() & is_df.NetIncomeLoss_hj.notnull())

is_df.loc[do_updated_opinc,'OperatingIncomeLoss_hj'] = is_df.loc[do_updated_opinc,'Revenues_hj'] \
                                    - avg_opinc * (is_df.loc[do_updated_opinc,'Revenues_hj'] - is_df.loc[do_updated_opinc,'NetIncomeLoss_hj'])

In [ ]:
(is_df.OperatingIncomeLoss_hj.isnull()).sum()

46

### estimate GrossProfit based on Revenue and OperatingIncomeLoss

In [ ]:
# if Grossprofit is not set, we can try to estimate it based on the average compared to reveneue and operatingIncomeLoss

# based on existing GrossProfit Values, we calculate the average grossprofit between Revenue and OperatingIncome (in percent)
calc_mean_gp_selection = ((is_df.Revenues_hj > 0.0)  & (is_df.GrossProfit.notnull()  & is_df.OperatingIncomeLoss_hj.notnull()))
selected_cols = is_df[calc_mean_gp_selection][['Revenues_hj','GrossProfit','OperatingIncomeLoss_hj']].copy()

avg_gross = ((selected_cols.Revenues_hj-selected_cols.GrossProfit)/(selected_cols.Revenues_hj-selected_cols.OperatingIncomeLoss_hj)).mean()

# if we have revenue and operatingincome, we try estimate grossprofit based on the average position
do_updated_grossprofit = (is_df.Revenues_hj.notnull() & is_df.OperatingIncomeLoss_hj.notnull() & is_df.GrossProfit.isnull())

is_df.loc[do_updated_grossprofit,'GrossProfit'] = is_df.loc[do_updated_grossprofit,'Revenues_hj'] \
                                    - avg_gross * (is_df.loc[do_updated_grossprofit,'Revenues_hj'] - is_df.loc[do_updated_grossprofit,'OperatingIncomeLoss_hj'])

In [ ]:
(is_df.Revenues_hj.isnull() | is_df.GrossProfit.isnull() | is_df.OperatingIncomeLoss_hj.isnull() | is_df.NetIncomeLoss_hj.isnull()).sum()

621

In [ ]:
fields = list(pivot_group)
fields.extend(['Revenues_hj','GrossProfit','OperatingIncomeLoss_hj','NetIncomeLoss_hj', 'SharesOutstanding_hj','EarningsPerShare_hj'])

### save

In [ ]:
is_df[fields].to_csv(all_data_local_folder + "05_is_completed.csv", index=False)

### remove companies with incomplete income statment

In [ ]:
incomplete_ciks = is_df[(is_df.Revenues_hj.isnull() | is_df.GrossProfit.isnull() | is_df.OperatingIncomeLoss_hj.isnull() | is_df.NetIncomeLoss_hj.isnull())].cik.unique()
print(len(incomplete_ciks))

160


In [ ]:
is_cleaned = is_df[~is_df.cik.isin(incomplete_ciks)]

In [ ]:
is_cleaned.shape

(111495, 21)

In [ ]:
is_cleaned[fields].to_csv(all_processed_folder + "is_clean.csv", index=False)

## XX_Trials

In [ ]:
is_df[((is_df.Revenues_hj.notnull() & is_df.GrossProfit.notnull() & is_df.OperatingIncomeLoss_hj.isnull() & is_df.NetIncomeLoss_hj.notnull() ))] \
  [['cik','adsh','Revenues_hj','GrossProfit','OperatingIncomeLoss_hj', 'NetIncomeLoss_hj', 'OperatingExpenses','CostsAndExpenses']] #721

In [ ]:
is_df[((is_df.Revenues_hj.isnull() | is_df.GrossProfit.isnull() | is_df.OperatingIncomeLoss_hj.isnull() | is_df.NetIncomeLoss_hj.isnull() ))] \
  [['cik','adsh','Revenues_hj','GrossProfit','OperatingIncomeLoss_hj', 'NetIncomeLoss_hj', 'OperatingExpenses','CostsAndExpenses']] #721

In [ ]:
is_df[((is_df.CostsAndExpenses.notnull() & is_df.GrossProfit.isnull() & is_df.OperatingIncomeLoss_hj.isnull()))][['cik','adsh','CostsAndExpenses', 'OperatingExpenses','GrossProfit','OperatingIncomeLoss_hj']] #721
#is_df[((is_df.OperatingExpenses.notnull() & is_df.GrossProfit.isnull() & is_df.OperatingIncomeLoss_hj.notnull()))][['cik','adsh','OperatingExpenses','GrossProfit','OperatingIncomeLoss_hj']] #11618

In [ ]:
is_df.isnull().sum(axis=0)

In [ ]:
is_df.adsh.value_counts()

In [ ]:
missing_value_report(cf_df)

In [ ]:
len(bs_df.adsh.unique()) == bs_df.shape[0]

True

In [ ]:
cf_df.dtypes

In [ ]:
sum(bs_df.qtrs>0)

1349


IS
Was kann man alles berechnen, wenn vorhanden
GrossProfit = Revenues - costofsales
OperatingIncome = (GrossProfit) - CostsAndExpenses
OperatingIncome = Revenues - CostsAndExpenses

In [ ]:
df = cf_df[(cf_df.null_count == 4)]
print(df.shape[0])
print(df.ticker.value_counts())